In [ ]:
import pandas as pd
import glob
import os
import zipfile
from io import TextIOWrapper

# 1월 따릉이 대여 반납 정보 CSV 파일 경로 가져오기
files = glob.glob("tpss_bcycl_od_statnhm_202401*.csv")

# 출발시간 열만 담을 리스트 생성
dfs = []
for file in files:
    df = pd.read_csv(file, encoding='cp949')
    startTime = df[df['집계_기준'] == '출발시간']
    dfs.append(startTime)

# 1일부터 31일 파일 하나로 합치기
result = pd.concat(dfs, ignore_index=True)

# 강수량 데이터 때문에 이 부분이 필요함
df = pd.read_csv('main.csv', encoding='cp949')
idAndgu = df[['대여소_ID', '구']].drop_duplicates()
result = result.merge(idAndgu, left_on='시작_대여소_ID', right_on='대여소_ID', how='left')

# 불필요 컬럼 삭제
result = result.drop(columns=[
    '기준_시간대', '종료_대여소명', '대여소_ID', '종료_대여소_ID', '집계_기준'
], errors='ignore')

# 강수량 정보
rainDf = pd.read_csv("2024 Rain Data.csv", encoding="cp949")
dfNoRain = rainDf[rainDf['일일강수량'] == 0]

dfNoRainCleaned = dfNoRain[
    (dfNoRain['측정소이름'] != '측정소이름') &
    (dfNoRain['일일강수량'] != '일일강수량')
]

# 조사년월일, 측정지역 컬럼만 남김
dfNoRainFinal = dfNoRainCleaned[['조사년월일', '측정지역']]
dfNoRainFinal['조사년월일'] = pd.to_datetime(dfNoRainFinal['조사년월일'], format='%Y%m%d')
rainZero = dfNoRainFinal[dfNoRainFinal['조사년월일'].dt.month == 1]
rainZero['조사년월일'] = rainZero['조사년월일'].dt.strftime('%Y%m%d')

result['기준_날짜'] = result['기준_날짜'].astype(str).str.zfill(8)
result = result.dropna(subset=['구'])
rainZero = rainZero.dropna(subset=['측정지역'])
result['구'] = result['구'].astype(str).str.strip()
rainZero['측정지역'] = rainZero['측정지역'].astype(str).str.strip()

result = result.merge(
    rainZero,
    left_on=['기준_날짜', '구'],
    right_on=['조사년월일', '측정지역'],
    how='inner'
).drop(columns=['조사년월일', '측정지역'])

# 평균, 총 사용인원
result['기준_날짜'] = pd.to_datetime(result['기준_날짜'], format='%Y%m%d')
result['월'] = result['기준_날짜'].dt.to_period('M')
result['일자'] = result['기준_날짜'].dt.date

totalUsers = result.groupby(['시작_대여소_ID', '월'])['전체_건수'].sum().reset_index(name='총_사용인원')
dayCounts = result.groupby(['시작_대여소_ID', '월'])['일자'].nunique().reset_index(name='일수')

use = pd.merge(totalUsers, dayCounts, on=['시작_대여소_ID', '월'])
use['평균_사용인원'] = use['총_사용인원'] / use['일수']

use = use.rename(columns={"시작_대여소_ID": "대여소_ID"})
use = use.drop(columns=[col for col in ['월', '일수'] if col in use.columns])

# 여기에 변수들만 모아져있는 파일 넣으면 될 것 같아요.
main = pd.read_csv("main.csv", encoding="cp949")
merged = pd.merge(main, use, how='left', on='대여소_ID')
merged.to_csv("mainJan.csv", index=False, encoding="cp949")

In [ ]:
# 2월 따릉이 대여 반납 정보 CSV 파일 경로 가져오기
files = glob.glob("tpss_bcycl_od_statnhm_202402*.csv")

# 출발시간 열만 담을 리스트 생성
dfs = []
for file in files:
    df = pd.read_csv(file, encoding='cp949')
    startTime = df[df['집계_기준'] == '출발시간']
    dfs.append(startTime)

# 1일부터 31일 파일 하나로 합치기
result = pd.concat(dfs, ignore_index=True)

# 강수량 데이터 때문에 이 부분이 필요함
df = pd.read_csv('main.csv', encoding='cp949')
idAndgu = df[['대여소_ID', '구']].drop_duplicates()
result = result.merge(idAndgu, left_on='시작_대여소_ID', right_on='대여소_ID', how='left')

# 불필요 컬럼 삭제
result = result.drop(columns=[
    '기준_시간대', '종료_대여소명', '대여소_ID', '종료_대여소_ID', '집계_기준'
], errors='ignore')

# 강수량 정보
rainDf = pd.read_csv("2024 Rain Data.csv", encoding="cp949")
dfNoRain = rainDf[rainDf['일일강수량'] == 0]

dfNoRainCleaned = dfNoRain[
    (dfNoRain['측정소이름'] != '측정소이름') &
    (dfNoRain['일일강수량'] != '일일강수량')
]

# 조사년월일, 측정지역 컬럼만 남김
dfNoRainFinal = dfNoRainCleaned[['조사년월일', '측정지역']]
dfNoRainFinal['조사년월일'] = pd.to_datetime(dfNoRainFinal['조사년월일'], format='%Y%m%d')
rainZero = dfNoRainFinal[dfNoRainFinal['조사년월일'].dt.month == 2]
rainZero['조사년월일'] = rainZero['조사년월일'].dt.strftime('%Y%m%d')

result['기준_날짜'] = result['기준_날짜'].astype(str).str.zfill(8)
result = result.dropna(subset=['구'])
rainZero = rainZero.dropna(subset=['측정지역'])
result['구'] = result['구'].astype(str).str.strip()
rainZero['측정지역'] = rainZero['측정지역'].astype(str).str.strip()

result = result.merge(
    rainZero,
    left_on=['기준_날짜', '구'],
    right_on=['조사년월일', '측정지역'],
    how='inner'
).drop(columns=['조사년월일', '측정지역'])

# 평균, 총 사용인원
result['기준_날짜'] = pd.to_datetime(result['기준_날짜'], format='%Y%m%d')
result['월'] = result['기준_날짜'].dt.to_period('M')
result['일자'] = result['기준_날짜'].dt.date

totalUsers = result.groupby(['시작_대여소_ID', '월'])['전체_건수'].sum().reset_index(name='총_사용인원')
dayCounts = result.groupby(['시작_대여소_ID', '월'])['일자'].nunique().reset_index(name='일수')

use = pd.merge(totalUsers, dayCounts, on=['시작_대여소_ID', '월'])
use['평균_사용인원'] = use['총_사용인원'] / use['일수']

use = use.rename(columns={"시작_대여소_ID": "대여소_ID"})
use = use.drop(columns=[col for col in ['월', '일수'] if col in use.columns])

# 최종 파일
main = pd.read_csv("main.csv", encoding="cp949")
merged = pd.merge(main, use, how='left', on='대여소_ID')
merged.to_csv("mainFeb.csv", index=False, encoding="cp949")

In [ ]:
# 3월 따릉이 대여 반납 정보 CSV 파일 경로 가져오기
files = glob.glob("tpss_bcycl_od_statnhm_202403*.csv")

# 출발시간 열만 담을 리스트 생성
dfs = []
for file in files:
    df = pd.read_csv(file, encoding='cp949')
    startTime = df[df['집계_기준'] == '출발시간']
    dfs.append(startTime) 

# 1일부터 31일 파일 하나로 합치기
result = pd.concat(dfs, ignore_index=True)

# 강수량 데이터 때문에 이 부분이 필요함
df = pd.read_csv('main.csv', encoding='cp949')
idAndgu = df[['대여소_ID', '구']].drop_duplicates()
result = result.merge(idAndgu, left_on='시작_대여소_ID', right_on='대여소_ID', how='left')

# 불필요 컬럼 삭제
result = result.drop(columns=[
    '기준_시간대', '종료_대여소명', '대여소_ID', '종료_대여소_ID', '집계_기준'
], errors='ignore')

# 강수량 정보
rainDf = pd.read_csv("2024 Rain Data.csv", encoding="cp949")
dfNoRain = rainDf[rainDf['일일강수량'] == 0]

dfNoRainCleaned = dfNoRain[
    (dfNoRain['측정소이름'] != '측정소이름') &
    (dfNoRain['일일강수량'] != '일일강수량')
]

# 조사년월일, 측정지역 컬럼만 남김
dfNoRainFinal = dfNoRainCleaned[['조사년월일', '측정지역']]
dfNoRainFinal['조사년월일'] = pd.to_datetime(dfNoRainFinal['조사년월일'], format='%Y%m%d')
rainZero = dfNoRainFinal[dfNoRainFinal['조사년월일'].dt.month == 3]
rainZero['조사년월일'] = rainZero['조사년월일'].dt.strftime('%Y%m%d')

result['기준_날짜'] = result['기준_날짜'].astype(str).str.zfill(8)
result = result.dropna(subset=['구'])
rainZero = rainZero.dropna(subset=['측정지역'])
result['구'] = result['구'].astype(str).str.strip()
rainZero['측정지역'] = rainZero['측정지역'].astype(str).str.strip()

result = result.merge(
    rainZero,
    left_on=['기준_날짜', '구'],
    right_on=['조사년월일', '측정지역'],
    how='inner'
).drop(columns=['조사년월일', '측정지역'])

# 평균, 총 사용인원
result['기준_날짜'] = pd.to_datetime(result['기준_날짜'], format='%Y%m%d')
result['월'] = result['기준_날짜'].dt.to_period('M')
result['일자'] = result['기준_날짜'].dt.date

totalUsers = result.groupby(['시작_대여소_ID', '월'])['전체_건수'].sum().reset_index(name='총_사용인원')
dayCounts = result.groupby(['시작_대여소_ID', '월'])['일자'].nunique().reset_index(name='일수')

use = pd.merge(totalUsers, dayCounts, on=['시작_대여소_ID', '월'])
use['평균_사용인원'] = use['총_사용인원'] / use['일수']

use = use.rename(columns={"시작_대여소_ID": "대여소_ID"})
use = use.drop(columns=[col for col in ['월', '일수'] if col in use.columns])

# 최종 파일
main = pd.read_csv("main.csv", encoding="cp949")
merged = pd.merge(main, use, how='left', on='대여소_ID')
merged.to_csv("mainMar.csv", index=False, encoding="cp949")

In [ ]:
# 4월 따릉이 대여 반납 정보 CSV 파일 경로 가져오기
files = glob.glob("tpss_bcycl_od_statnhm_202404*.csv")

# 출발시간 열만 담을 리스트 생성
dfs = []
for file in files:
    df = pd.read_csv(file, encoding='cp949')
    startTime = df[df['집계_기준'] == '출발시간']
    dfs.append(startTime) 

# 1일부터 31일 파일 하나로 합치기
result = pd.concat(dfs, ignore_index=True)

# 강수량 데이터 때문에 이 부분이 필요함
df = pd.read_csv('main.csv', encoding='cp949')
idAndgu = df[['대여소_ID', '구']].drop_duplicates()
result = result.merge(idAndgu, left_on='시작_대여소_ID', right_on='대여소_ID', how='left')

# 불필요 컬럼 삭제
result = result.drop(columns=[
    '기준_시간대', '종료_대여소명', '대여소_ID', '종료_대여소_ID', '집계_기준'
], errors='ignore')

# 강수량 정보
rainDf = pd.read_csv("2024 Rain Data.csv", encoding="cp949")
dfNoRain = rainDf[rainDf['일일강수량'] == 0]

dfNoRainCleaned = dfNoRain[
    (dfNoRain['측정소이름'] != '측정소이름') &
    (dfNoRain['일일강수량'] != '일일강수량')
]

# 조사년월일, 측정지역 컬럼만 남김
dfNoRainFinal = dfNoRainCleaned[['조사년월일', '측정지역']]
dfNoRainFinal['조사년월일'] = pd.to_datetime(dfNoRainFinal['조사년월일'], format='%Y%m%d')
rainZero = dfNoRainFinal[dfNoRainFinal['조사년월일'].dt.month == 4]
rainZero['조사년월일'] = rainZero['조사년월일'].dt.strftime('%Y%m%d')

result['기준_날짜'] = result['기준_날짜'].astype(str).str.zfill(8)
result = result.dropna(subset=['구'])
rainZero = rainZero.dropna(subset=['측정지역'])
result['구'] = result['구'].astype(str).str.strip()
rainZero['측정지역'] = rainZero['측정지역'].astype(str).str.strip()

result = result.merge(
    rainZero,
    left_on=['기준_날짜', '구'],
    right_on=['조사년월일', '측정지역'],
    how='inner'
).drop(columns=['조사년월일', '측정지역'])

# 평균, 총 사용인원
result['기준_날짜'] = pd.to_datetime(result['기준_날짜'], format='%Y%m%d')
result['월'] = result['기준_날짜'].dt.to_period('M')
result['일자'] = result['기준_날짜'].dt.date

totalUsers = result.groupby(['시작_대여소_ID', '월'])['전체_건수'].sum().reset_index(name='총_사용인원')
dayCounts = result.groupby(['시작_대여소_ID', '월'])['일자'].nunique().reset_index(name='일수')

use = pd.merge(totalUsers, dayCounts, on=['시작_대여소_ID', '월'])
use['평균_사용인원'] = use['총_사용인원'] / use['일수']

use = use.rename(columns={"시작_대여소_ID": "대여소_ID"})
use = use.drop(columns=[col for col in ['월', '일수'] if col in use.columns])

# 최종 파일
main = pd.read_csv("main.csv", encoding="cp949")
merged = pd.merge(main, use, how='left', on='대여소_ID')
merged.to_csv("mainApr.csv", index=False, encoding="cp949")

In [ ]:
# 5월 따릉이 대여 반납 정보 CSV 파일 경로 가져오기
files = glob.glob("tpss_bcycl_od_statnhm_202405*.csv")

# 출발시간 열만 담을 리스트 생성
dfs = []
for file in files:
    df = pd.read_csv(file, encoding='cp949')
    startTime = df[df['집계_기준'] == '출발시간']
    dfs.append(startTime) 

# 1일부터 31일 파일 하나로 합치기
result = pd.concat(dfs, ignore_index=True)

# 강수량 데이터 때문에 이 부분이 필요함
df = pd.read_csv('main.csv', encoding='cp949')
idAndgu = df[['대여소_ID', '구']].drop_duplicates()
result = result.merge(idAndgu, left_on='시작_대여소_ID', right_on='대여소_ID', how='left')

# 불필요 컬럼 삭제
result = result.drop(columns=[
    '기준_시간대', '종료_대여소명', '대여소_ID', '종료_대여소_ID', '집계_기준'
], errors='ignore')

# 강수량 정보
rainDf = pd.read_csv("2024 Rain Data.csv", encoding="cp949")
dfNoRain = rainDf[rainDf['일일강수량'] == 0]

dfNoRainCleaned = dfNoRain[
    (dfNoRain['측정소이름'] != '측정소이름') &
    (dfNoRain['일일강수량'] != '일일강수량')
]

# 조사년월일, 측정지역 컬럼만 남김
dfNoRainFinal = dfNoRainCleaned[['조사년월일', '측정지역']]
dfNoRainFinal['조사년월일'] = pd.to_datetime(dfNoRainFinal['조사년월일'], format='%Y%m%d')
rainZero = dfNoRainFinal[dfNoRainFinal['조사년월일'].dt.month == 5]
rainZero['조사년월일'] = rainZero['조사년월일'].dt.strftime('%Y%m%d')

result['기준_날짜'] = result['기준_날짜'].astype(str).str.zfill(8)
result = result.dropna(subset=['구'])
rainZero = rainZero.dropna(subset=['측정지역'])
result['구'] = result['구'].astype(str).str.strip()
rainZero['측정지역'] = rainZero['측정지역'].astype(str).str.strip()

result = result.merge(
    rainZero,
    left_on=['기준_날짜', '구'],
    right_on=['조사년월일', '측정지역'],
    how='inner'
).drop(columns=['조사년월일', '측정지역'])

# 평균, 총 사용인원
result['기준_날짜'] = pd.to_datetime(result['기준_날짜'], format='%Y%m%d')
result['월'] = result['기준_날짜'].dt.to_period('M')
result['일자'] = result['기준_날짜'].dt.date

totalUsers = result.groupby(['시작_대여소_ID', '월'])['전체_건수'].sum().reset_index(name='총_사용인원')
dayCounts = result.groupby(['시작_대여소_ID', '월'])['일자'].nunique().reset_index(name='일수')

use = pd.merge(totalUsers, dayCounts, on=['시작_대여소_ID', '월'])
use['평균_사용인원'] = use['총_사용인원'] / use['일수']

use = use.rename(columns={"시작_대여소_ID": "대여소_ID"})
use = use.drop(columns=[col for col in ['월', '일수'] if col in use.columns])

# 최종 파일
main = pd.read_csv("main.csv", encoding="cp949")
merged = pd.merge(main, use, how='left', on='대여소_ID')
merged.to_csv("mainMay.csv", index=False, encoding="cp949")

In [ ]:
# 6월 따릉이 대여 반납 정보 CSV 파일 경로 가져오기
files = glob.glob("tpss_bcycl_od_statnhm_202406*.csv")

# 출발시간 열만 담을 리스트 생성
dfs = []
for file in files:
    df = pd.read_csv(file, encoding='cp949')
    startTime = df[df['집계_기준'] == '출발시간']
    dfs.append(startTime)

# 1일부터 31일 파일 하나로 합치기
result = pd.concat(dfs, ignore_index=True)

# 강수량 데이터 때문에 이 부분이 필요함
df = pd.read_csv('main.csv', encoding='cp949')
idAndgu = df[['대여소_ID', '구']].drop_duplicates()
result = result.merge(idAndgu, left_on='시작_대여소_ID', right_on='대여소_ID', how='left')

# 불필요 컬럼 삭제
result = result.drop(columns=[
    '기준_시간대', '종료_대여소명', '대여소_ID', '종료_대여소_ID', '집계_기준'
], errors='ignore')

# 강수량 정보
rainDf = pd.read_csv("2024 Rain Data.csv", encoding="cp949")
dfNoRain = rainDf[rainDf['일일강수량'] == 0]

dfNoRainCleaned = dfNoRain[
    (dfNoRain['측정소이름'] != '측정소이름') &
    (dfNoRain['일일강수량'] != '일일강수량')
]

# 조사년월일, 측정지역 컬럼만 남김
dfNoRainFinal = dfNoRainCleaned[['조사년월일', '측정지역']]
dfNoRainFinal['조사년월일'] = pd.to_datetime(dfNoRainFinal['조사년월일'], format='%Y%m%d')
rainZero = dfNoRainFinal[dfNoRainFinal['조사년월일'].dt.month == 6]
rainZero['조사년월일'] = rainZero['조사년월일'].dt.strftime('%Y%m%d')

result['기준_날짜'] = result['기준_날짜'].astype(str).str.zfill(8)
result = result.dropna(subset=['구'])
rainZero = rainZero.dropna(subset=['측정지역'])
result['구'] = result['구'].astype(str).str.strip()
rainZero['측정지역'] = rainZero['측정지역'].astype(str).str.strip()

result = result.merge(
    rainZero,
    left_on=['기준_날짜', '구'],
    right_on=['조사년월일', '측정지역'],
    how='inner'
).drop(columns=['조사년월일', '측정지역'])

# 평균, 총 사용인원
result['기준_날짜'] = pd.to_datetime(result['기준_날짜'], format='%Y%m%d')
result['월'] = result['기준_날짜'].dt.to_period('M')
result['일자'] = result['기준_날짜'].dt.date

totalUsers = result.groupby(['시작_대여소_ID', '월'])['전체_건수'].sum().reset_index(name='총_사용인원')
dayCounts = result.groupby(['시작_대여소_ID', '월'])['일자'].nunique().reset_index(name='일수')

use = pd.merge(totalUsers, dayCounts, on=['시작_대여소_ID', '월'])
use['평균_사용인원'] = use['총_사용인원'] / use['일수']

use = use.rename(columns={"시작_대여소_ID": "대여소_ID"})
use = use.drop(columns=[col for col in ['월', '일수'] if col in use.columns])

# 최종 파일
main = pd.read_csv("main.csv", encoding="cp949")
merged = pd.merge(main, use, how='left', on='대여소_ID')
merged.to_csv("mainJun.csv", index=False, encoding="cp949")

In [ ]:
# 7월 따릉이 대여 반납 정보 CSV 파일 경로 가져오기
files = glob.glob("tpss_bcycl_od_statnhm_202407*.csv")

# 출발시간 열만 담을 리스트 생성
dfs = []
for file in files:
    df = pd.read_csv(file, encoding='cp949')
    startTime = df[df['집계_기준'] == '출발시간']
    dfs.append(startTime) 

# 1일부터 31일 파일 하나로 합치기
result = pd.concat(dfs, ignore_index=True)

# 강수량 데이터 때문에 이 부분이 필요함
df = pd.read_csv('main.csv', encoding='cp949')
idAndgu = df[['대여소_ID', '구']].drop_duplicates()
result = result.merge(idAndgu, left_on='시작_대여소', right_on='대여소_ID', how='left')

# 불필요 컬럼 삭제
result = result.drop(columns=[
    '기준_시간대', '종료_대여소명', '대여소_ID', '종료_대여소_ID', '집계_기준'
], errors='ignore')

# 강수량 정보
rainDf = pd.read_csv("2024 Rain Data.csv", encoding="cp949")
dfNoRain = rainDf[rainDf['일일강수량'] == 0]

dfNoRainCleaned = dfNoRain[
    (dfNoRain['측정소이름'] != '측정소이름') &
    (dfNoRain['일일강수량'] != '일일강수량')
]

# 조사년월일, 측정지역 컬럼만 남김
dfNoRainFinal = dfNoRainCleaned[['조사년월일', '측정지역']]
dfNoRainFinal['조사년월일'] = pd.to_datetime(dfNoRainFinal['조사년월일'], format='%Y%m%d')
rainZero = dfNoRainFinal[dfNoRainFinal['조사년월일'].dt.month == 7]
rainZero['조사년월일'] = rainZero['조사년월일'].dt.strftime('%Y%m%d')

result['기준_날짜'] = result['기준_날짜'].astype(str).str.zfill(8)
result = result.dropna(subset=['구'])
rainZero = rainZero.dropna(subset=['측정지역'])
result['구'] = result['구'].astype(str).str.strip()
rainZero['측정지역'] = rainZero['측정지역'].astype(str).str.strip()

result = result.merge(
    rainZero,
    left_on=['기준_날짜', '구'],
    right_on=['조사년월일', '측정지역'],
    how='inner'
).drop(columns=['조사년월일', '측정지역'])

# 평균, 총 사용인원
result['기준_날짜'] = pd.to_datetime(result['기준_날짜'], format='%Y%m%d')
result['월'] = result['기준_날짜'].dt.to_period('M')
result['일자'] = result['기준_날짜'].dt.date

totalUsers = result.groupby(['시작_대여소', '월'])['전체_건수'].sum().reset_index(name='총_사용인원')
dayCounts = result.groupby(['시작_대여소', '월'])['일자'].nunique().reset_index(name='일수')

use = pd.merge(totalUsers, dayCounts, on=['시작_대여소', '월'])
use['평균_사용인원'] = use['총_사용인원'] / use['일수']

use = use.rename(columns={"시작_대여소": "대여소_ID"})
use = use.drop(columns=[col for col in ['월', '일수'] if col in use.columns])

# 최종 파일
main = pd.read_csv("main.csv", encoding="cp949")
merged = pd.merge(main, use, how='left', on='대여소_ID')
merged.to_csv("mainJul.csv", index=False, encoding="cp949")

In [ ]:
# 8월 따릉이 대여 반납 정보 CSV 파일 경로 가져오기
files = glob.glob("tpss_bcycl_od_statnhm_202408*.csv")

# 출발시간 열만 담을 리스트 생성
dfs = []
for file in files:
    df = pd.read_csv(file, encoding='cp949')
    startTime = df[df['집계_기준'] == '출발시간']
    dfs.append(startTime) 

# 1일부터 31일 파일 하나로 합치기
result = pd.concat(dfs, ignore_index=True)

# 강수량 데이터 때문에 이 부분이 필요함
df = pd.read_csv('main.csv', encoding='cp949')
idAndgu = df[['대여소_ID', '구']].drop_duplicates()
result = result.merge(idAndgu, left_on='시작_대여소_ID', right_on='대여소_ID', how='left')

# 불필요 컬럼 삭제
result = result.drop(columns=[
    '기준_시간대', '종료_대여소명', '대여소_ID', '종료_대여소_ID', '집계_기준'
], errors='ignore')

# 강수량 정보
rainDf = pd.read_csv("2024 Rain Data.csv", encoding="cp949")
dfNoRain = rainDf[rainDf['일일강수량'] == 0]

dfNoRainCleaned = dfNoRain[
    (dfNoRain['측정소이름'] != '측정소이름') &
    (dfNoRain['일일강수량'] != '일일강수량')
]

# 조사년월일, 측정지역 컬럼만 남김
dfNoRainFinal = dfNoRainCleaned[['조사년월일', '측정지역']]
dfNoRainFinal['조사년월일'] = pd.to_datetime(dfNoRainFinal['조사년월일'], format='%Y%m%d')
rainZero = dfNoRainFinal[dfNoRainFinal['조사년월일'].dt.month == 8]
rainZero['조사년월일'] = rainZero['조사년월일'].dt.strftime('%Y%m%d')

result['기준_날짜'] = result['기준_날짜'].astype(str).str.zfill(8)
result = result.dropna(subset=['구'])
rainZero = rainZero.dropna(subset=['측정지역'])
result['구'] = result['구'].astype(str).str.strip()
rainZero['측정지역'] = rainZero['측정지역'].astype(str).str.strip()

result = result.merge(
    rainZero,
    left_on=['기준_날짜', '구'],
    right_on=['조사년월일', '측정지역'],
    how='inner'
).drop(columns=['조사년월일', '측정지역'])

# 평균, 총 사용인원
result['기준_날짜'] = pd.to_datetime(result['기준_날짜'], format='%Y%m%d')
result['월'] = result['기준_날짜'].dt.to_period('M')
result['일자'] = result['기준_날짜'].dt.date

totalUsers = result.groupby(['시작_대여소_ID', '월'])['전체_건수'].sum().reset_index(name='총_사용인원')
dayCounts = result.groupby(['시작_대여소_ID', '월'])['일자'].nunique().reset_index(name='일수')

use = pd.merge(totalUsers, dayCounts, on=['시작_대여소_ID', '월'])
use['평균_사용인원'] = use['총_사용인원'] / use['일수']

use = use.rename(columns={"시작_대여소_ID": "대여소_ID"})
use = use.drop(columns=[col for col in ['월', '일수'] if col in use.columns])

# 최종 파일
main = pd.read_csv("main.csv", encoding="cp949")
merged = pd.merge(main, use, how='left', on='대여소_ID')
merged.to_csv("mainAug.csv", index=False, encoding="cp949")

In [ ]:
# 9월 따릉이 대여 반납 정보 CSV 파일 경로 가져오기
files = glob.glob("tpss_bcycl_od_statnhm_202409*.csv")

# 출발시간 열만 담을 리스트 생성
dfs = []
for file in files:
    df = pd.read_csv(file, encoding='cp949')
    startTime = df[df['집계_기준'] == '출발시간']
    dfs.append(startTime) 

# 1일부터 31일 파일 하나로 합치기
result = pd.concat(dfs, ignore_index=True)

# 강수량 데이터 때문에 이 부분이 필요함
df = pd.read_csv('main.csv', encoding='cp949')
idAndgu = df[['대여소_ID', '구']].drop_duplicates()
result = result.merge(idAndgu, left_on='시작_대여소_ID', right_on='대여소_ID', how='left')

# 불필요 컬럼 삭제
result = result.drop(columns=[
    '기준_시간대', '종료_대여소명', '대여소_ID', '종료_대여소_ID', '집계_기준'
], errors='ignore')

# 강수량 정보
rainDf = pd.read_csv("2024 Rain Data.csv", encoding="cp949")
dfNoRain = rainDf[rainDf['일일강수량'] == 0]

dfNoRainCleaned = dfNoRain[
    (dfNoRain['측정소이름'] != '측정소이름') &
    (dfNoRain['일일강수량'] != '일일강수량')
]

# 조사년월일, 측정지역 컬럼만 남김
dfNoRainFinal = dfNoRainCleaned[['조사년월일', '측정지역']]
dfNoRainFinal['조사년월일'] = pd.to_datetime(dfNoRainFinal['조사년월일'], format='%Y%m%d')
rainZero = dfNoRainFinal[dfNoRainFinal['조사년월일'].dt.month == 9]
rainZero['조사년월일'] = rainZero['조사년월일'].dt.strftime('%Y%m%d')

result['기준_날짜'] = result['기준_날짜'].astype(str).str.zfill(8)
result = result.dropna(subset=['구'])
rainZero = rainZero.dropna(subset=['측정지역'])
result['구'] = result['구'].astype(str).str.strip()
rainZero['측정지역'] = rainZero['측정지역'].astype(str).str.strip()

result = result.merge(
    rainZero,
    left_on=['기준_날짜', '구'],
    right_on=['조사년월일', '측정지역'],
    how='inner'
).drop(columns=['조사년월일', '측정지역'])

# 평균, 총 사용인원
result['기준_날짜'] = pd.to_datetime(result['기준_날짜'], format='%Y%m%d')
result['월'] = result['기준_날짜'].dt.to_period('M')
result['일자'] = result['기준_날짜'].dt.date

totalUsers = result.groupby(['시작_대여소_ID', '월'])['전체_건수'].sum().reset_index(name='총_사용인원')
dayCounts = result.groupby(['시작_대여소_ID', '월'])['일자'].nunique().reset_index(name='일수')

use = pd.merge(totalUsers, dayCounts, on=['시작_대여소_ID', '월'])
use['평균_사용인원'] = use['총_사용인원'] / use['일수']

use = use.rename(columns={"시작_대여소_ID": "대여소_ID"})
use = use.drop(columns=[col for col in ['월', '일수'] if col in use.columns])

# 최종 파일
main = pd.read_csv("main.csv", encoding="cp949")
merged = pd.merge(main, use, how='left', on='대여소_ID')
merged.to_csv("mainSep.csv", index=False, encoding="cp949")

In [ ]:
# 10월 따릉이 대여 반납 정보 CSV 파일 경로 가져오기
files = glob.glob("tpss_bcycl_od_statnhm_202410*.csv")

# 출발시간 열만 담을 리스트 생성
dfs = []
for file in files:
    df = pd.read_csv(file, encoding='cp949')
    startTime = df[df['집계_기준'] == '출발시간']
    dfs.append(startTime) 

# 1일부터 31일 파일 하나로 합치기
result = pd.concat(dfs, ignore_index=True)

# 강수량 데이터 때문에 이 부분이 필요함
df = pd.read_csv('main.csv', encoding='cp949')
idAndgu = df[['대여소_ID', '구']].drop_duplicates()
result = result.merge(idAndgu, left_on='시작_대여소_ID', right_on='대여소_ID', how='left')

# 불필요 컬럼 삭제
result = result.drop(columns=[
    '기준_시간대', '종료_대여소명', '대여소_ID', '종료_대여소_ID', '집계_기준'
], errors='ignore')

# 강수량 정보
rainDf = pd.read_csv("2024 Rain Data.csv", encoding="cp949")
dfNoRain = rainDf[rainDf['일일강수량'] == 0]

dfNoRainCleaned = dfNoRain[
    (dfNoRain['측정소이름'] != '측정소이름') &
    (dfNoRain['일일강수량'] != '일일강수량')
]

# 조사년월일, 측정지역 컬럼만 남김
dfNoRainFinal = dfNoRainCleaned[['조사년월일', '측정지역']]
dfNoRainFinal['조사년월일'] = pd.to_datetime(dfNoRainFinal['조사년월일'], format='%Y%m%d')
rainZero = dfNoRainFinal[dfNoRainFinal['조사년월일'].dt.month == 10]
rainZero['조사년월일'] = rainZero['조사년월일'].dt.strftime('%Y%m%d')

result['기준_날짜'] = result['기준_날짜'].astype(str).str.zfill(8)
result = result.dropna(subset=['구'])
rainZero = rainZero.dropna(subset=['측정지역'])
result['구'] = result['구'].astype(str).str.strip()
rainZero['측정지역'] = rainZero['측정지역'].astype(str).str.strip()

result = result.merge(
    rainZero,
    left_on=['기준_날짜', '구'],
    right_on=['조사년월일', '측정지역'],
    how='inner'
).drop(columns=['조사년월일', '측정지역'])

# 평균, 총 사용인원
result['기준_날짜'] = pd.to_datetime(result['기준_날짜'], format='%Y%m%d')
result['월'] = result['기준_날짜'].dt.to_period('M')
result['일자'] = result['기준_날짜'].dt.date

totalUsers = result.groupby(['시작_대여소_ID', '월'])['전체_건수'].sum().reset_index(name='총_사용인원')
dayCounts = result.groupby(['시작_대여소_ID', '월'])['일자'].nunique().reset_index(name='일수')

use = pd.merge(totalUsers, dayCounts, on=['시작_대여소_ID', '월'])
use['평균_사용인원'] = use['총_사용인원'] / use['일수']

use = use.rename(columns={"시작_대여소_ID": "대여소_ID"})
use = use.drop(columns=[col for col in ['월', '일수'] if col in use.columns])

# 최종 파일
main = pd.read_csv("main.csv", encoding="cp949")
merged = pd.merge(main, use, how='left', on='대여소_ID')
merged.to_csv("mainOct.csv", index=False, encoding="cp949")

In [ ]:
# 11월 따릉이 대여 반납 정보 CSV 파일 경로 가져오기
files = glob.glob("tpss_bcycl_od_statnhm_202411*.csv")

# 출발시간 열만 담을 리스트 생성
dfs = []
for file in files:
    df = pd.read_csv(file, encoding='cp949')
    startTime = df[df['집계_기준'] == '출발시간']
    dfs.append(startTime) 

# 1일부터 31일 파일 하나로 합치기
result = pd.concat(dfs, ignore_index=True)

# 강수량 데이터 때문에 이 부분이 필요함
df = pd.read_csv('main.csv', encoding='cp949')
idAndgu = df[['대여소_ID', '구']].drop_duplicates()
result = result.merge(idAndgu, left_on='시작_대여소_ID', right_on='대여소_ID', how='left')

# 불필요 컬럼 삭제
result = result.drop(columns=[
    '기준_시간대', '종료_대여소명', '대여소_ID', '종료_대여소_ID', '집계_기준'
], errors='ignore')

# 강수량 정보
rainDf = pd.read_csv("2024 Rain Data.csv", encoding="cp949")
dfNoRain = rainDf[rainDf['일일강수량'] == 0]

dfNoRainCleaned = dfNoRain[
    (dfNoRain['측정소이름'] != '측정소이름') &
    (dfNoRain['일일강수량'] != '일일강수량')
]

# 조사년월일, 측정지역 컬럼만 남김
dfNoRainFinal = dfNoRainCleaned[['조사년월일', '측정지역']]
dfNoRainFinal['조사년월일'] = pd.to_datetime(dfNoRainFinal['조사년월일'], format='%Y%m%d')
rainZero = dfNoRainFinal[dfNoRainFinal['조사년월일'].dt.month == 11]
rainZero['조사년월일'] = rainZero['조사년월일'].dt.strftime('%Y%m%d')

result['기준_날짜'] = result['기준_날짜'].astype(str).str.zfill(8)
result = result.dropna(subset=['구'])
rainZero = rainZero.dropna(subset=['측정지역'])
result['구'] = result['구'].astype(str).str.strip()
rainZero['측정지역'] = rainZero['측정지역'].astype(str).str.strip()

result = result.merge(
    rainZero,
    left_on=['기준_날짜', '구'],
    right_on=['조사년월일', '측정지역'],
    how='inner'
).drop(columns=['조사년월일', '측정지역'])

# 평균, 총 사용인원
result['기준_날짜'] = pd.to_datetime(result['기준_날짜'], format='%Y%m%d')
result['월'] = result['기준_날짜'].dt.to_period('M')
result['일자'] = result['기준_날짜'].dt.date

totalUsers = result.groupby(['시작_대여소_ID', '월'])['전체_건수'].sum().reset_index(name='총_사용인원')
dayCounts = result.groupby(['시작_대여소_ID', '월'])['일자'].nunique().reset_index(name='일수')

use = pd.merge(totalUsers, dayCounts, on=['시작_대여소_ID', '월'])
use['평균_사용인원'] = use['총_사용인원'] / use['일수']

use = use.rename(columns={"시작_대여소_ID": "대여소_ID"})
use = use.drop(columns=[col for col in ['월', '일수'] if col in use.columns])

# 최종 파일
main = pd.read_csv("main.csv", encoding="cp949")
merged = pd.merge(main, use, how='left', on='대여소_ID')
merged.to_csv("mainNov.csv", index=False, encoding="cp949")

In [ ]:
# 12월 따릉이 대여 반납 정보 CSV 파일 경로 가져오기
files = glob.glob("tpss_bcycl_od_statnhm_202412*.csv")

# 출발시간 열만 담을 리스트 생성
dfs = []
for file in files:
    df = pd.read_csv(file, encoding='cp949')
    startTime = df[df['집계_기준'] == '출발시간']
    dfs.append(startTime) 

# 1일부터 31일 파일 하나로 합치기
result = pd.concat(dfs, ignore_index=True)

# 강수량 데이터 때문에 이 부분이 필요함
df = pd.read_csv('main.csv', encoding='cp949')
idAndgu = df[['대여소_ID', '구']].drop_duplicates()
result = result.merge(idAndgu, left_on='시작_대여소_ID', right_on='대여소_ID', how='left')

# 불필요 컬럼 삭제
result = result.drop(columns=[
    '기준_시간대', '종료_대여소명', '대여소_ID', '종료_대여소_ID', '집계_기준'
], errors='ignore')

# 강수량 정보
rainDf = pd.read_csv("2024 Rain Data.csv", encoding="cp949")
dfNoRain = rainDf[rainDf['일일강수량'] == 0]

dfNoRainCleaned = dfNoRain[
    (dfNoRain['측정소이름'] != '측정소이름') &
    (dfNoRain['일일강수량'] != '일일강수량')
]

# 조사년월일, 측정지역 컬럼만 남김
dfNoRainFinal = dfNoRainCleaned[['조사년월일', '측정지역']]
dfNoRainFinal['조사년월일'] = pd.to_datetime(dfNoRainFinal['조사년월일'], format='%Y%m%d')
rainZero = dfNoRainFinal[dfNoRainFinal['조사년월일'].dt.month == 12]
rainZero['조사년월일'] = rainZero['조사년월일'].dt.strftime('%Y%m%d')

result['기준_날짜'] = result['기준_날짜'].astype(str).str.zfill(8)
result = result.dropna(subset=['구'])
rainZero = rainZero.dropna(subset=['측정지역'])
result['구'] = result['구'].astype(str).str.strip()
rainZero['측정지역'] = rainZero['측정지역'].astype(str).str.strip()

result = result.merge(
    rainZero,
    left_on=['기준_날짜', '구'],
    right_on=['조사년월일', '측정지역'],
    how='inner'
).drop(columns=['조사년월일', '측정지역'])

# 평균, 총 사용인원
result['기준_날짜'] = pd.to_datetime(result['기준_날짜'], format='%Y%m%d')
result['월'] = result['기준_날짜'].dt.to_period('M')
result['일자'] = result['기준_날짜'].dt.date

totalUsers = result.groupby(['시작_대여소_ID', '월'])['전체_건수'].sum().reset_index(name='총_사용인원')
dayCounts = result.groupby(['시작_대여소_ID', '월'])['일자'].nunique().reset_index(name='일수')

use = pd.merge(totalUsers, dayCounts, on=['시작_대여소_ID', '월'])
use['평균_사용인원'] = use['총_사용인원'] / use['일수']

use = use.rename(columns={"시작_대여소_ID": "대여소_ID"})
use = use.drop(columns=[col for col in ['월', '일수'] if col in use.columns])

# 최종 파일
main = pd.read_csv("main.csv", encoding="cp949")
merged = pd.merge(main, use, how='left', on='대여소_ID')
merged.to_csv("mainDec.csv", index=False, encoding="cp949")

In [ ]:
# mainJan.csv ~ mainDec.csv 압축
files = [f'main{month}.csv' for month in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
                                          'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']]

zipFileName = 'month total.zip'

with zipfile.ZipFile(zipFileName, 'w') as zipf:
    for file in files:
        if os.path.exists(file):
            zipf.write(file)
        else:
            print(f"{file} 파일이 존재하지 않음")

print("압축 완료했습니다.")
